<a href="https://colab.research.google.com/github/sirrice/joinboost-demo/blob/main/demo/JoinBoostRandomForest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install JoinBoost and databases package.
In this demo, we use duckdb as the database.


In [ ]:
%pip install joinboost==0.0.1521
%pip install duckdb

# Download data and load data into database.

This step is only necessary for this demo as we don't have existing databases. JoinBoost to directly build models over your databases without data downloading/loading.

In [ ]:
def CreateJoinGraph(exe=None):
  dataset = JoinGraph(exe=exe)
  dataset.add_relation("sales", [], y = 'Y')
  dataset.add_relation("holidays", ["htype", "locale", "locale_name", "transferred","f2"])
  dataset.add_relation("oil", ["dcoilwtico","f3"])
  dataset.add_relation("transactions", ["transactions","f5"])
  dataset.add_relation("stores", ["city","state","stype","cluster","f4"])
  dataset.add_relation("items", ["family","class","perishable","f1"])
  dataset.add_join("sales", "items", ["item_nbr"], ["item_nbr"])
  dataset.add_join("sales", "transactions", ["tid"], ["tid"])
  dataset.add_join("transactions", "stores", ["store_nbr"], ["store_nbr"])
  dataset.add_join("transactions", "holidays", ["date"], ["date"])
  dataset.add_join("holidays", "oil", ["date"], ["date"])
  return dataset

In [ ]:
import duckdb
import time
import urllib.request
from joinboost.executor import DuckdbExecutor
from joinboost.joingraph import JoinGraph
from joinboost.app import DecisionTree,GradientBoosting,RandomForest

urllib.request.urlretrieve("https://www.dropbox.com/s/kaovdndtevcvt83/holidays.csv?dl=1", "holidays.csv")
urllib.request.urlretrieve("https://www.dropbox.com/s/wh6amz4um7ieyqz/items.csv?dl=1", "items.csv")
urllib.request.urlretrieve("https://www.dropbox.com/s/ze6of1xqwslt8jb/oil.csv?dl=1", "oil.csv")
urllib.request.urlretrieve("https://www.dropbox.com/s/lcqpncj48ds3jao/sales.parquet?dl=1", "sales.parquet")
urllib.request.urlretrieve("https://www.dropbox.com/s/cwy6z0b7rhsnrxb/stores.csv?dl=1", "stores.csv")
urllib.request.urlretrieve("https://www.dropbox.com/s/2uogid62tc95fwa/test.csv?dl=1", "test.csv")
urllib.request.urlretrieve("https://www.dropbox.com/s/2bxto9wnetwnvqd/transactions.csv?dl=1", "transactions.csv")

In [ ]:
con = duckdb.connect(database=':memory:')
con.execute("CREATE OR REPLACE TABLE holidays AS SELECT * FROM 'holidays.csv';")
con.execute("CREATE OR REPLACE TABLE oil AS SELECT * FROM 'oil.csv';")
con.execute("CREATE OR REPLACE TABLE transactions AS SELECT * FROM 'transactions.csv';")
con.execute("CREATE OR REPLACE TABLE stores AS SELECT * FROM 'stores.csv';")
con.execute("CREATE OR REPLACE TABLE items AS SELECT * FROM 'items.csv';")
con.execute("CREATE OR REPLACE TABLE sales AS SELECT * FROM 'sales.parquet';")
con.execute("CREATE OR REPLACE TABLE test AS SELECT * FROM 'test.csv';")

# Data exploration and Data Transformation
You can use the database SQL queries to show all available tables and their schema. You can perform data transformations over these tables.

In [ ]:
con.execute("describe;").df()

In [ ]:
con.execute("select count(*) from sales;").df()

# Build Join Graph



In [ ]:
exe = DuckdbExecutor(con, debug=False)
dataset = CreateJoinGraph(exe=exe)
dataset

# XGboost

In [ ]:
%%time
join = con.execute("""
SELECT 	Y,onpromotion,htype,locale,locale_name,transferred,f2,oil.date AS date,dcoilwtico,f3,sales.tid AS tid,transactions,f5,stores.store_nbr AS store_nbr,city,state,stype,cluster,f4,sales.item_nbr AS item_nbr,family,class,perishable,f1,unit_sales
FROM sales
JOIN items ON items.item_nbr = sales.item_nbr
JOIN transactions ON sales.tid = transactions.tid
JOIN stores ON transactions.store_nbr = stores.store_nbr
JOIN holidays ON transactions.date = holidays.date
JOIN oil ON holidays.date = oil.date
""").df()

In [ ]:
%%time
import xgboost as xgb
x = ["onpromotion","htype","locale","locale_name","transferred","f2","date","dcoilwtico","f3","tid","transactions","f5","store_nbr","city","state","stype","cluster","f4","item_nbr","family","class","perishable","f1","unit_sales"]
y = "Y"
xgb_model = xgb.XGBRFRegressor(max_leaves=8, max_depth=3, n_estimators=5, learning_rate=1, subsample=1, colsample_bynode=1, reg_lambda=0)
xgb_model.fit(join[x], join[y])


In [ ]:
from sklearn.metrics import mean_squared_error
test = con.execute("select * from test;").df()
print(f"rmse {mean_squared_error(test[y], xgb_model.predict(test[x]))**0.5:.6f}")

# JoinBoost

In [ ]:
%%time
reg = RandomForest(max_leaves=8, max_depth=3, subsample=1,iteration=5)
reg.fit(dataset)

In [ ]:
print(f"rmse {reg.compute_rmse('test')[0]:.6f}")

# JoinBoost Internals
Internally, JoinBoost translates ML logics into SQL, and execute SQL queries directly in databases.

In [ ]:
dataset.exe.debug=True
reg = RandomForest(max_leaves=8, max_depth=3, subsample=1,iteration=1)
reg.fit(dataset)